In [6]:
# Load libraries
import pandas as pd
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import numpy as np
import gensim
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model
from sklearn.linear_model import LinearRegression


model = gensim.models.word2vec.Word2Vec.load("word2vec.model")

TESTING_MODE = False

# load dataset
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
test_data = test_data.drop("index",axis=1)
data = pd.concat([train_data,test_data],sort=False)

In [2]:
if TESTING_MODE:
    d = pd.get_dummies(train_data[["country","region_1","region_2","province","taster_name","variety"]])
    d["points"]= train_data["points"]
    d["id"] = train_data['id']
else:
    d = pd.get_dummies(data[["country","region_1","region_2","province","taster_name","variety"]])
    d["points"]= data["points"]
    d["id"] = data['id']

In [3]:
description_vecs = train_data["description"].apply(lambda words:sum([model[word.lower()] for word in words.split()])*1/len(words.split()))

for i in range(300):
    d["vec_dim_"+str(i)]=description_vecs.apply(lambda v:v[i])

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [4]:
if TESTING_MODE:
    X_train, X_test, y_train, y_test = train_test_split(d, train_data["price"], test_size=0.3, random_state=1) # 70% training and 30% test
else:
    X_train,y_train,X_test, y_test = d[:175000], data["price"][:175000], d[175000:], data["price"][175000:]

In [11]:
len(data["designation"].unique())

44390

In [5]:
X_train = X_train.drop(["id"],axis=1)

In [7]:
%%time

#Edit: added second square bracket above to fix the ValueError problem

poly = PolynomialFeatures(degree=2)
X_ = poly.fit_transform(X_train)
# predict_ = poly.fit_transform(predict)

clf = LinearRegression()
clf.fit(X_, y_train)

# # Create Decision Tree classifer object
# clf = RandomForestClassifier(100,n_jobs=20)
# # Train Decision Tree Classifer
# clf = clf.fit(X_train,y_train)

MemoryError: 

In [ ]:
#Predict the response for test dataset
ids = X_test["id"]
X_test = X_test.drop(["id"],axis=1)

y_pred = clf.predict(X_test)

In [ ]:
print(y_test[:10],y_pred[:10])

In [ ]:
np.argmax(y_pred)

In [ ]:
result = pd.DataFrame()
result['id'] = ids
result["price"] = y_pred

result["price"]=result["price"].apply(int)

result.to_csv('out_rf.csv',index=False)

In [ ]:
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))